In [5]:
import pandas as pd
import numpy as np
import os
import logging
from pathlib import Path
from yfpy import YahooFantasySportsQuery
from yfpy.utils import complex_json_handler
from yfpy import get_logger
from dotenv import load_dotenv
from ratelimit import limits, sleep_and_retry
from gspread_pandas import Spread

from utils import (
    get_season,
    league_season_info,
    google_sheet_upload,
    sql_upload_table,
    sql_grab_table,
)


load_dotenv()
pd.set_option("display.max_colwidth", 999)
pd.set_option("display.max_columns", 999)
LOGGET = get_logger(__name__)
LOG_OUTPUT = False
logging.getLogger("yfpy.query").setLevel(level=logging.INFO)

PATH = Path.cwd().parents[0]
DATA_DIR = PATH / "data"

SEASON = get_season()
NFL_DATES_DF, LEAGUE_ID_DF = league_season_info()

# TODAY = np.datetime64("today", "D")
TODAY = np.datetime64("2021-09-28")
NFL_WEEK = NFL_DATES_DF["Week"][
    (NFL_DATES_DF["End_Date"] >= TODAY) & (NFL_DATES_DF["Start_Date"] <= TODAY)
].values[0]
LEAGUE_ID = LEAGUE_ID_DF[LEAGUE_ID_DF["season"] == SEASON]["league_ID"].values[0]
GAME_ID = LEAGUE_ID_DF[LEAGUE_ID_DF["season"] == SEASON]["game_ID"].values[0]
WEEKS = list(range(NFL_WEEK, 0, -1))

CONSUMER_KEY = os.getenv("yahoo_client_id")
CONSUMER_SECRET = os.getenv("yahoo_client_secret")

try:
    yahoo_query = YahooFantasySportsQuery(
        auth_dir=PATH,
        league_id=LEAGUE_ID,
        game_id=GAME_ID,
        game_code="nfl",
        offline=False,
        all_output_as_json=False,
        consumer_key=CONSUMER_KEY,
        consumer_secret=CONSUMER_SECRET,
        browser_callback=True,
    )


except Exception as e:
    print(e)

In [6]:
class league_season_data(object):

    rate_limited_time = 3
    rate_limited_calls = 1

    def __init__(
        self,
        auth_dir: Path,
        league_id: str,
        game_id: int = None,
        game_code: str = "nfl",
        offline: bool = False,
        all_output_as_json: bool = False,
        consumer_key: str = None,
        consumer_secret: str = None,
        browser_callback: bool = True,
    ):

        self._auth_dir = auth_dir
        self._consumer_key = consumer_key
        self._consumer_secret = consumer_secret
        self._browser_callback = browser_callback

        self.league_id = league_id
        self.game_id = game_id
        self.game_code = game_code

        self.offline = offline
        self.all_output_as_json = all_output_as_json

    def _yahoo_query(self):

        try:
            yahoo_query = YahooFantasySportsQuery(
                auth_dir=self._auth_dir,
                league_id=self.league_id,
                game_id=self.game_id,
                game_code=self.game_code,
                offline=self.offline,
                all_output_as_json=self.all_output_as_json,
                consumer_key=self._consumer_key,
                consumer_secret=self._consumer_secret,
                browser_callback=self._browser_callback,
            )

            return yahoo_query

        except Exception as e:
            print(e)

    @sleep_and_retry
    @limits(calls=rate_limited_calls, period=rate_limited_time)
    def get_league_metadata(self, first_time='no'):
        response = complex_json_handler(yahoo_query.get_league_metadata())
        league_metadata = pd.json_normalize(response)

        league_metadata["game_id"] = self.game_id

        if str(first_time).upper() == 'YES':
            sql_upload_table(
                dataframe=league_metadata,
                table_name="LeagueMetaData",
                data_schema="dbo",
                chunksize=500,
                if_exists="replace",
                index=False,
            )

        elif str(first_time).upper() == 'NO':
            sql_upload_table(
                dataframe=league_metadata,
                table_name="LeagueMetaData",
                data_schema="dbo",
                chunksize=500,
                if_exists="append",
                index=False,
            )

        return league_metadata

    @sleep_and_retry
    @limits(calls=rate_limited_calls, period=rate_limited_time)
    def get_league_set_roster_pos_stat_cat(self, first_time='no'):
        response = complex_json_handler(yahoo_query.get_league_settings())

        league_settings = pd.json_normalize(response)
        league_settings.drop(
            ["roster_positions", "stat_categories.stats", "stat_modifiers.stats"],
            axis=1,
            inplace=True,
        )

        league_settings["game_id"] = self.game_id
        league_settings["league_id"] = self.league_id

        roster_positions = pd.DataFrame()
        for r in response["roster_positions"]:
            row = pd.json_normalize(complex_json_handler(r["roster_position"]))
            roster_positions = pd.concat([roster_positions, row])

        roster_positions["game_id"] = self.game_id
        roster_positions["league_id"] = self.league_id

        stat_categories = pd.DataFrame()
        for r in response["stat_categories"]["stats"]:
            row = pd.json_normalize(complex_json_handler(r["stat"]))
            row["position_type"] = complex_json_handler(
                complex_json_handler(r["stat"])["stat_position_types"][
                    "stat_position_type"
                ]
            )["position_type"]
            try:
                row["is_only_display_stat"] = complex_json_handler(
                    complex_json_handler(r["stat"])["stat_position_types"][
                        "stat_position_type"
                    ]
                )["is_only_display_stat"]
            except:
                row["is_only_display_stat"] = 0
            row.drop("stat_position_types.stat_position_type", axis=1, inplace=True)
            stat_categories = pd.concat([stat_categories, row])

        stat_categories["game_id"] = self.game_id
        stat_categories["league_id"] = self.league_id

        stat_modifiers = pd.DataFrame()
        for r in response["stat_modifiers"]["stats"]:
            row = pd.json_normalize(complex_json_handler(r["stat"]))
            stat_modifiers = pd.concat([stat_modifiers, row])

        stat_modifiers.rename(columns={"value": "stat_modifier"}, inplace=True)

        stat_categories = stat_categories.merge(
            stat_modifiers, how="outer", on="stat_id"
        )

        if str(first_time).upper() == 'YES':
            sql_upload_table(
                dataframe=league_settings,
                table_name="LeagueSettings",
                data_schema="dbo",
                chunksize=500,
                if_exists="replace",
                index=False,
            )

            sql_upload_table(
                dataframe=roster_positions,
                table_name="RosterPositions",
                data_schema="dbo",
                chunksize=500,
                if_exists="replace",
                index=False,
            )

            sql_upload_table(
                dataframe=stat_categories,
                table_name="StatCategories",
                data_schema="dbo",
                chunksize=500,
                if_exists="replace",
                index=False,
            )

        elif str(first_time).upper() == 'NO':
            sql_upload_table(
                dataframe=league_settings,
                table_name="LeagueSettings",
                data_schema="dbo",
                chunksize=500,
                if_exists="append",
                index=False,
            )

            sql_upload_table(
                dataframe=roster_positions,
                table_name="RosterPositions",
                data_schema="dbo",
                chunksize=500,
                if_exists="append",
                index=False,
            )

            sql_upload_table(
                dataframe=stat_categories,
                table_name="StatCategories",
                data_schema="dbo",
                chunksize=500,
                if_exists="append",
                index=False,
            )
            
        return league_settings, roster_positions, stat_categories

    @sleep_and_retry
    @limits(calls=rate_limited_calls, period=rate_limited_time)
    def get_league_teams_list(self, first_time='no'):
        response = yahoo_query.get_league_teams()
        teams = pd.DataFrame()
        for r in response:
            team = pd.json_normalize(complex_json_handler(r["team"]))
            manager = pd.json_normalize(
                complex_json_handler(team["managers.manager"][0])
            )
            team = pd.concat([team, manager], axis=1)
            if "clinched_playoffs" not in team.columns:
                team["clinched_playoffs"] = 0
            teams = pd.concat([teams, team], ignore_index=True).reset_index(drop=True)
            teams.drop(
                ["url", "managers.manager", "team_logos.team_logo", "image_url"],
                axis=1,
                inplace=True,
            )

        teams["name"] = teams["name"].str.decode("utf-8")

        teams["game_id"] = self.game_id
        teams["league_id"] = self.league_id

        if str(first_time).upper() == 'YES':
            sql_upload_table(
                dataframe=teams,
                table_name="Teams",
                data_schema="dbo",
                chunksize=500,
                if_exists="replace",
                index=False,
            )

        elif str(first_time).upper() == 'NO':
            sql_upload_table(
                dataframe=teams,
                table_name="Teams",
                data_schema="dbo",
                chunksize=500,
                if_exists="append",
                index=False,
            )

        return teams

    @sleep_and_retry
    @limits(calls=rate_limited_calls, period=rate_limited_time)
    def get_league_players_list(self, first_time='no'):
        response = yahoo_query.get_league_players()
        players = pd.DataFrame()
        for r in response:
            player = pd.json_normalize(complex_json_handler(r["player"]))
            draft_analysis = pd.json_normalize(
                complex_json_handler(
                    yahoo_query.get_player_draft_analysis(player["player_key"][0])
                )
            )
            draft_analysis = draft_analysis[
                [
                    "draft_analysis.average_pick",
                    "draft_analysis.average_round",
                    "draft_analysis.average_cost",
                    "draft_analysis.percent_drafted",
                ]
            ]
            player = pd.concat([player, draft_analysis], axis=1)
            if "status" not in player.columns:
                player["status"] = np.nan

            players = pd.concat([players, player], ignore_index=True)

            players["game_id"] = self.game_id
            players["league_id"] = self.league_id

        if str(first_time).upper() == 'YES':
            sql_upload_table(
                dataframe=players,
                table_name="Players",
                data_schema="dbo",
                chunksize=500,
                if_exists="replace",
                index=False,
            )

        elif str(first_time).upper() == 'NO':
            sql_upload_table(
                dataframe=players,
                table_name="Players",
                data_schema="dbo",
                chunksize=500,
                if_exists="append",
                index=False,
            )
            
        return players

    @sleep_and_retry
    @limits(calls=rate_limited_calls, period=rate_limited_time)
    def get_league_draft_results(self, first_time='no'):
        response = yahoo_query.get_league_draft_results()
        draft_results = pd.DataFrame()
        for r in response:
            row = pd.json_normalize(complex_json_handler(r["draft_result"]))
            draft_results = pd.concat([draft_results, row])

        draft_results["game_id"] = self.game_id
        draft_results["league_id"] = self.league_id

        if str(first_time).upper() == 'YES':
            sql_upload_table(
                dataframe=draft_results,
                table_name="DraftResults",
                data_schema="dbo",
                chunksize=500,
                if_exists="replace",
                index=False,
            )

        elif str(first_time).upper() == 'NO':
            sql_upload_table(
                dataframe=draft_results,
                table_name="DraftResults",
                data_schema="dbo",
                chunksize=500,
                if_exists="append",
                index=False,
            )
            
        return draft_results

    @sleep_and_retry
    @limits(calls=rate_limited_calls, period=rate_limited_time)
    def get_league_matchups_by_week(self, first_time='no'):
        m = []
        i = 0
        while True:
            try:
                response = yahoo_query.get_league_matchups_by_week(i)
                if bool(response) == True:
                    for data in response:
                        m.append(complex_json_handler(data["matchup"]))
                    i += 1
                else:
                    break
            except:
                break
            
        matchups = pd.DataFrame()
        for r in m:
            matchup = pd.json_normalize(r)
            team_a = pd.json_normalize(
                complex_json_handler(r["matchup_grades"][0]["matchup_grade"])
            )
            team_a["points"] = complex_json_handler(r["teams"][0]["team"])[
                "team_points"
            ]["total"]
            team_a["projected_points"] = complex_json_handler(r["teams"][0]["team"])[
                "team_projected_points"
            ]["total"]
            team_a = team_a.add_prefix("team_a_")
            team_b = pd.json_normalize(
                complex_json_handler(r["matchup_grades"][1]["matchup_grade"])
            )
            team_b["points"] = complex_json_handler(r["teams"][1]["team"])[
                "team_points"
            ]["total"]
            team_b["projected_points"] = complex_json_handler(r["teams"][1]["team"])[
                "team_projected_points"
            ]["total"]
            team_b = team_b.add_prefix("team_b_")
            matchup = pd.concat([matchup, team_a, team_b], axis=1)

            matchups = pd.concat([matchups, matchup])

        matchups.drop(["teams", "matchup_grades"], axis=1, inplace=True)

        matchups["game_id"] = self.game_id
        matchups["league_id"] = self.league_id

        if str(first_time).upper() == 'YES':
            sql_upload_table(
                dataframe=matchups,
                table_name="Matchups",
                data_schema="dbo",
                chunksize=500,
                if_exists="replace",
                index=False,
            )

        elif str(first_time).upper() == 'NO':
            sql_upload_table(
                dataframe=matchups,
                table_name="Matchups",
                data_schema="dbo",
                chunksize=500,
                if_exists="append",
                index=False,
            )

        return matchups

    @sleep_and_retry
    @limits(calls=rate_limited_calls, period=rate_limited_time)
    def get_team_standings(self, first_time='no'):
        standings = pd.DataFrame()
        i = 1
        while True:
            try:
                response = yahoo_query.get_team_standings(i)
                if bool(response) == True:
                    row = pd.json_normalize(complex_json_handler(response))
                    row["team_id"] = i
                    standings = pd.concat([standings, row])
                    i += 1
                else:
                    break
            except:
                break

        standings["game_id"] = self.game_id
        standings["league_id"] = self.league_id

        if str(first_time).upper() == 'YES':
            sql_upload_table(
                dataframe=standings,
                table_name="Standings",
                data_schema="dbo",
                chunksize=500,
                if_exists="replace",
                index=False,
            )

        elif str(first_time).upper() == 'NO':
            sql_upload_table(
                dataframe=standings,
                table_name="Standings",
                data_schema="dbo",
                chunksize=500,
                if_exists="append",
                index=False,
            )

        return standings

    @sleep_and_retry
    @limits(calls=rate_limited_calls, period=rate_limited_time)
    def get_team_roster_by_week(self, first_time='no'):
        team_rosters = pd.DataFrame()
        t = 1
        w = 1
        for t in range(1, 12):
            try:
                for w in range(1, 18):
                    try:
                        response = complex_json_handler(
                            yahoo_query.get_team_roster_by_week(t, w)
                        )
                        if bool(response) == True:
                            row = pd.json_normalize(
                                complex_json_handler(response["players"][0]["player"])
                            )
                            row["team_id"] = t
                            row["week"] = w
                            team_rosters = pd.concat([team_rosters, row])
                        else:
                            break
                    except:
                        break
            except:
                break

        team_rosters["game_id"] = self.game_id
        team_rosters["league_id"] = self.league_id

        if str(first_time).upper() == 'YES':
            sql_upload_table(
                dataframe=team_rosters,
                table_name="TeamRosters",
                data_schema="dbo",
                chunksize=500,
                if_exists="replace",
                index=False,
            )

        elif str(first_time).upper() == 'NO':
            sql_upload_table(
                dataframe=team_rosters,
                table_name="TeamRosters",
                data_schema="dbo",
                chunksize=500,
                if_exists="append",
                index=False,
            )

        return team_rosters

    @sleep_and_retry
    @limits(calls=rate_limited_calls, period=rate_limited_time)
    def get_player_stats_by_week(self):
        pass

    @sleep_and_retry
    @limits(calls=rate_limited_calls, period=rate_limited_time)
    def get_player_percent_owned_by_week(self):
        pass

    @sleep_and_retry
    @limits(calls=rate_limited_calls, period=rate_limited_time)
    def all_game_keys(self):
        response = complex_json_handler(yahoo_query.get_all_yahoo_fantasy_game_keys())
        game_keys = pd.DataFrame()
        for r in response:
            row = pd.DataFrame(complex_json_handler(r["game"]), index=[0])
            game_keys = pd.concat([game_keys, row])

        game_keys.reset_index(drop=True, inplace=True)
        game_keys = game_keys[game_keys["season"] >= 2012]

        sql_upload_table(
            dataframe=game_keys,
            table_name="GameKeys",
            data_schema="dbo",
            chunksize=500,
            if_exists="replace",
            index=False,
        )

        return game_keys

    @sleep_and_retry
    @limits(calls=rate_limited_calls, period=rate_limited_time)
    def all_nfl_weeks(self):
        game_keys = sql_grab_table("GameKeys")
        game_id = list(game_keys["game_id"])
        weeks = pd.DataFrame()
        for g in game_id:
            response = yahoo_query.get_game_weeks_by_game_id(g)
            for r in response:
                row = pd.json_normalize(complex_json_handler(r["game_week"]))
                row["game_id"] = g
                weeks = pd.concat([weeks, row])

        weeks.rename(columns={"display_name": "week"}, inplace=True)

        sql_upload_table(
            dataframe=weeks,
            table_name="NFLWeeks",
            data_schema="dbo",
            chunksize=500,
            if_exists="replace",
            index=False,
        )

        return weeks

In [7]:
league = league_season_data(
    auth_dir=PATH,
    league_id=LEAGUE_ID,
    game_id=GAME_ID,
    game_code="nfl",
    offline=False,
    all_output_as_json=False,
    consumer_key=CONSUMER_KEY,
    consumer_secret=CONSUMER_SECRET,
    browser_callback=True,
)

In [8]:
players = league.get_league_players_list()

HTTPError: Yahoo data unavailable due to rate limiting. Please try again later.